<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/tf_quant_finance_notebooks/hull_white_calibration_from_cap_floors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## tf_quant_finance.models.hull_white.calibration_from_swaptions

The Hull-White one-factor model assumes that the short rate $r(t)$ evolves according to the stochastic differential equation:
$$
dr(t) = \left[\theta(t) - a r(t)\right] dt + \sigma dW(t),
$$
where:
- $a$: mean-reversion speed,
- $\sigma$: volatility of the short rate,
- $\theta(t)$: time-dependent drift, calibrated to fit the initial term structure,
- $W(t)$: standard Brownian motion under the risk-neutral measure.

This function estimates the mean-reversion rate and volatility parameters of a Hull-White 1-factor model using a set of European swaption prices as the target. The calibration is performed using least-squares optimization where the loss function minimizes the squared error between the target swaption prices and the model implied swaption prices.

### Example
The example shows how to calibrate a Hull-White model with constant mean reversion rate and constant volatility.

In [1]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

In [2]:
!pip install tf-quant-finance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 2.4 MB/s eta 0:00:00


In [3]:
import numpy as np
import tensorflow.compat.v2 as tf
import tf_quant_finance as tff

# In this example, we synthetically generate some prices. Then we use our
# calibration to back out these prices.
dtype = tf.float64

daycount_fractions = np.array([
    [0.25, 0.25, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0],
    [0.25, 0.25, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0],
    [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
    [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25],
])
expiries = np.array([
    [0.0, 0.25, 0.5, 0.75, 1.0, 0.0, 0.0, 0.0],
    [0.0, 0.25, 0.5, 0.75, 1.0, 0.0, 0.0, 0.0],
    [0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.50, 1.75],
    [0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.50, 1.75],
])
maturities = np.array([
    [0.25, 0.5, 0.75, 1.0, 0.0, 0.0, 0.0, 0.0],
    [0.25, 0.5, 0.75, 1.0, 0.0, 0.0, 0.0, 0.0],
    [0.25, 0.5, 0.75, 1.0, 1.25, 1.50, 1.75, 2.0],
    [0.25, 0.5, 0.75, 1.0, 1.25, 1.50, 1.75, 2.0],
])
is_cap = np.array([True, False, True, False])
strikes = 0.01 * np.ones_like(expiries)

# Setup - generate some observed prices using the model.
expected_mr = [0.4]
expected_vol = [0.01]

zero_rate_fn = lambda x: 0.01 * tf.ones_like(x, dtype=dtype)
prices = tff.models.hull_white.cap_floor_price(
    strikes=strikes,
    expiries=expiries,
    maturities=maturities,
    daycount_fractions=daycount_fractions,
    reference_rate_fn=zero_rate_fn,
    notional=1.0,
    mean_reversion=expected_mr,
    volatility=expected_vol,
    is_cap=tf.expand_dims(is_cap, axis=1),
    use_analytic_pricing=True,
    dtype=dtype)

# Calibrate the model.
calibrated_model, is_converged, _ = (
    tff.models.hull_white.calibration_from_cap_floors(
        prices=tf.squeeze(prices),
        strikes=strikes,
        expiries=expiries,
        maturities=maturities,
        daycount_fractions=daycount_fractions,
        reference_rate_fn=zero_rate_fn,
        mean_reversion=[0.3],
        volatility=[0.02],
        notional=1.0,
        is_cap=tf.expand_dims(is_cap, axis=1),
        use_analytic_pricing=True,
        optimizer_fn=None,
        num_samples=1000,
        random_type=tff.math.random.RandomType.STATELESS_ANTITHETIC,
        seed=[0, 0],
        time_step=0.1,
        maximum_iterations=200,
        dtype=dtype))

calibrated_mr = calibrated_model.mean_reversion.values()
calibrated_vol = calibrated_model.volatility.values()

# Running this inside a unit test passes:
#
# calibrated_mr, calibrated_vol = self.evaluate(
#     [calibrated_mr, calibrated_vol])
# self.assertTrue(is_converged)
# self.assertAllClose(calibrated_mr, expected_mr, atol=1e-3, rtol=1e-2)
# self.assertAllClose(calibrated_vol, expected_vol, atol=1e-3, rtol=1e-2)

In [4]:
calibrated_mr, calibrated_vol

(<tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.40052417])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.01000292])>)